In [2]:
import pandas as pd
import sqlite3

df = pd.read_csv('india_export.csv')
display(df)

,HSCode,Commodity,value,country,year
0,2,MEAT AND EDIBLE MEAT OFFAL.,0.18,AFGHANISTAN TIS,2018
1,3,"FISH AND CRUSTACEANS, MOLLUSCS AND OTHER AQUAT...",0.00,AFGHANISTAN TIS,2018
2,4,DAIRY PRODUCE; BIRDS' EGGS; NATURAL HONEY; EDI...,12.48,AFGHANISTAN TIS,2018
3,6,LIVE TREES AND OTHER PLANTS; BULBS; ROOTS AND ...,0.00,AFGHANISTAN TIS,2018
4,7,EDIBLE VEGETABLES AND CERTAIN ROOTS AND TUBERS.,1.89,AFGHANISTAN TIS,2018
...,...,...,...,...,...
137018,95,"TOYS, GAMES AND SPORTS REQUISITES; PARTS AND A...",0.03,ZIMBABWE,2010
137019,96,MISCELLANEOUS MANUFACTURED ARTICLES.,0.45,ZIMBABWE,2010
137020,97,WORKS OF ART COLLECTORS' PIECES AND ANTIQUES.,0.00,ZIMBABWE,2010
137021,98,PROJECT GOODS; SOME SPECIAL USES.,0.00,ZIMBABWE,2010


In [3]:
conn = sqlite3.connect('india_export.db')
cur = conn.cursor()
cur.execute('''DROP TABLE IF EXISTS export''')
df.to_sql('export', conn, if_exists='replace', index=False) # - writes the pd.df to SQLIte DB
conn.commit()

In [4]:
pd.read_sql('select * from export', conn)

,HSCode,Commodity,value,country,year
0,2,MEAT AND EDIBLE MEAT OFFAL.,0.18,AFGHANISTAN TIS,2018
1,3,"FISH AND CRUSTACEANS, MOLLUSCS AND OTHER AQUAT...",0.00,AFGHANISTAN TIS,2018
2,4,DAIRY PRODUCE; BIRDS' EGGS; NATURAL HONEY; EDI...,12.48,AFGHANISTAN TIS,2018
3,6,LIVE TREES AND OTHER PLANTS; BULBS; ROOTS AND ...,0.00,AFGHANISTAN TIS,2018
4,7,EDIBLE VEGETABLES AND CERTAIN ROOTS AND TUBERS.,1.89,AFGHANISTAN TIS,2018
...,...,...,...,...,...
137018,95,"TOYS, GAMES AND SPORTS REQUISITES; PARTS AND A...",0.03,ZIMBABWE,2010
137019,96,MISCELLANEOUS MANUFACTURED ARTICLES.,0.45,ZIMBABWE,2010
137020,97,WORKS OF ART COLLECTORS' PIECES AND ANTIQUES.,0.00,ZIMBABWE,2010
137021,98,PROJECT GOODS; SOME SPECIAL USES.,0.00,ZIMBABWE,2010


In [5]:
conn.close()

# Steps to normalizing India export database

- Inspect table and determine how many tables into convert the database into 
    - Four tables:
        - year 
        - country   
        - commodity 
        - a table that joins country, year, commmodity, and value 
        
- Steps to create year table
    - list of unique years
    - create statement
    - insert function 


In [6]:
import pandas as pd
import sqlite3
from sqlite3 import Error

def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()

    return rows


In [7]:
conn = create_connection('india_export.db')

In [9]:
# Get list of unique years

sql_statement = "SELECT DISTINCT year from export ORDER BY year"

years = execute_sql_statement(sql_statement, conn)
years = list(map(lambda row: int(row[0]), years))
print(years)

[2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]


In [11]:
create_table_sql = """CREATE TABLE [Year] (
    [YEAR] INTEGER NOT NULL PRIMARY KEY
);
"""
# conn_norm.close()

conn_norm = create_connection('india_export_norm.db', True)
create_table(conn_norm, create_table_sql)
sql_statement = "SELECT * FROM Year"
df = pd.read_sql_query(sql_statement, conn_norm)
display(df)
conn_norm.close()

,YEAR


In [13]:
conn_norm = create_connection('india_export_norm.db')
sql_statement = "SELECT * FROM Year"
df = pd.read_sql_query(sql_statement, conn_norm)
display(df)

,YEAR


In [14]:
(2022,)

(2022,)

In [15]:
def insert_year(conn, values):
    sql = ''' INSERT INTO YEAR(YEAR)
              VALUES(?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

with conn_norm:
    for year in years:
        insert_year(conn_norm, (year, ))


In [16]:
sql_statement = "SELECT * FROM Year"
df = pd.read_sql_query(sql_statement, conn_norm)
display(df)

,YEAR
0,2010
1,2011
2,2012
3,2013
4,2014
5,2015
6,2016
7,2017
8,2018


In [19]:
conn_norm.close()


In [18]:
conn_norm = create_connection('india_export_norm.db')
sql_statement = "SELECT * FROM Year"
df = pd.read_sql_query(sql_statement, conn_norm)
display(df)

,YEAR
0,2010
1,2011
2,2012
3,2013
4,2014
5,2015
6,2016
7,2017
8,2018


In [20]:
# Get list of unique countries

sql_statement = "SELECT DISTINCT country from export ORDER BY country"

countries = execute_sql_statement(sql_statement, conn)
countries = list(map(lambda row: row[0], countries))
print(countries)

['AFGHANISTAN TIS', 'ALBANIA', 'ALGERIA', 'AMERI SAMOA', 'ANDORRA', 'ANGOLA', 'ANGUILLA', 'ANTARTICA', 'ANTIGUA', 'ARGENTINA', 'ARMENIA', 'ARUBA', 'AUSTRALIA', 'AUSTRIA', 'AZERBAIJAN', 'BAHAMAS', 'BAHARAIN IS', 'BANGLADESH PR', 'BARBADOS', 'BELARUS', 'BELGIUM', 'BELIZE', 'BENIN', 'BERMUDA', 'BHUTAN', 'BOLIVIA', 'BOSNIA-HRZGOVIN', 'BOTSWANA', 'BR VIRGN IS', 'BRAZIL', 'BRUNEI', 'BULGARIA', 'BURKINA FASO', 'BURUNDI', 'C AFRI REP', 'CAMBODIA', 'CAMEROON', 'CANADA', 'CANARY IS', 'CAPE VERDE IS', 'CAYMAN IS', 'CHAD', 'CHANNEL IS', 'CHILE', 'CHINA P RP', 'CHRISTMAS IS.', 'COCOS IS', 'COLOMBIA', 'COMOROS', 'CONGO D. REP.', 'CONGO P REP', 'COOK IS', 'COSTA RICA', "COTE D' IVOIRE", 'CROATIA', 'CUBA', 'CURACAO', 'CYPRUS', 'CZECH REPUBLIC', 'DENMARK', 'DJIBOUTI', 'DOMINIC REP', 'DOMINICA', 'ECUADOR', 'EGYPT A RP', 'EL SALVADOR', 'EQUTL GUINEA', 'ERITREA', 'ESTONIA', 'ETHIOPIA', 'FALKLAND IS', 'FAROE IS.', 'FIJI IS', 'FINLAND', 'FR GUIANA', 'FR POLYNESIA', 'FR S ANT TR', 'FRANCE', 'GABON', 'GAMBIA'

In [22]:
def insert_country(conn, values):
    sql = ''' INSERT INTO COUNTRY(COUNTRY)
              VALUES(?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

create_table_sql = """CREATE TABLE IF NOT EXISTS [Country] (
    [Country] TEXT NOT NULL PRIMARY KEY
);
"""
conn_norm = create_connection('india_export_norm.db')

create_table(conn_norm, create_table_sql)

with conn_norm:
    for country in countries:
        insert_country(conn_norm, (country, ))
        
sql_statement = "SELECT * FROM country"
df = pd.read_sql_query(sql_statement, conn_norm)
display(df)


,Country
0,AFGHANISTAN TIS
1,ALBANIA
2,ALGERIA
3,AMERI SAMOA
4,ANDORRA
...,...
243,VIRGIN IS US
244,WALLIS F IS
245,YEMEN REPUBLC
246,ZAMBIA


In [23]:
sql_statement = "SELECT DISTINCT COMMODITY FROM export order by COMMODITY"
commodities = execute_sql_statement(sql_statement, conn)
print(commodities)
print(len(commodities))

[('AIRCRAFT, SPACECRAFT, AND PARTS THEREOF.',), ('ALBUMINOIDAL SUBSTANCES; MODIFIED STARCHES; GLUES; ENZYMES.',), ('ALUMINIUM AND ARTICLES THEREOF.',), ('ANIMAL OR VEGETABLE FATS AND OILS AND THEIR CLEAVAGE PRODUCTS; PRE. EDIBLE FATS; ANIMAL OR VEGETABLE WAXEX.',), ('ARMS AND AMMUNITION; PARTS AND ACCESSORIES THEREOF.',), ('ARTICLES OF APPAREL AND CLOTHING ACCESSORIES, KNITTED OR CORCHETED.',), ('ARTICLES OF APPAREL AND CLOTHING ACCESSORIES, NOT KNITTED OR CROCHETED.',), ('ARTICLES OF IRON OR STEEL',), ('ARTICLES OF LEATHER,SADDLERY AND HARNESS;TRAVEL GOODS, HANDBAGS AND SIMILAR CONT.ARTICLES OF ANIMAL GUT(OTHR THN SILK-WRM)GUT.',), ('ARTICLES OF STONE, PLASTER, CEMENT, ASBESTOS, MICA OR SIMILAR MATERIALS.',), ('BEVERAGES, SPIRITS AND VINEGAR.',), ('CARPETS AND OTHER TEXTILE FLOOR COVERINGS.',), ('CERAMIC PRODUCTS.',), ('CEREALS.',), ('CLOCKS AND WATCHES AND PARTS THEREOF.',), ('COCOA AND COCOA PREPARATIONS.',), ('COFFEE, TEA, MATE AND SPICES.',), ('COPPER AND ARTICLES THEREOF.',), ('C

In [24]:
def insert_commodity(conn, values):
    sql = ''' INSERT INTO COMMODITY (COMMODITY)
              VALUES(?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

create_table_sql = """CREATE TABLE IF NOT EXISTS [COMMODITY] (
    [HSCODE] INTEGER NOT NULL PRIMARY KEY, 
    [COMMODITY] TEXT NOT NULL
);
"""
create_table(conn_norm, create_table_sql)



In [25]:
with conn_norm:
    for commodity in commodities:
        insert_commodity(conn_norm, commodity)
        
sql_statement = "SELECT * FROM Commodity"
df = pd.read_sql_query(sql_statement, conn_norm)
display(df)

,HSCODE,COMMODITY
0,1,"AIRCRAFT, SPACECRAFT, AND PARTS THEREOF."
1,2,ALBUMINOIDAL SUBSTANCES; MODIFIED STARCHES; GL...
2,3,ALUMINIUM AND ARTICLES THEREOF.
3,4,ANIMAL OR VEGETABLE FATS AND OILS AND THEIR CL...
4,5,ARMS AND AMMUNITION; PARTS AND ACCESSORIES THE...
...,...,...
93,94,"WADDING, FELT AND NONWOVENS; SPACIAL YARNS; TW..."
94,95,WOOD AND ARTICLES OF WOOD; WOOD CHARCOAL.
95,96,"WOOL, FINE OR COARSE ANIMAL HAIR, HORSEHAIR YA..."
96,97,WORKS OF ART COLLECTORS' PIECES AND ANTIQUES.


In [26]:
# Table 4 which is going link all the tables with values

def insert_value(conn, values):
    sql = ''' INSERT INTO Export (HSCODE, COUNTRY, YEAR, VALUE)
              VALUES(?, ?, ?, ?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

create_table_sql = """CREATE TABLE IF NOT EXISTS [Export] (
    [VALUEID] INTEGER NOT NULL PRIMARY KEY,
    [HSCODE] INTEGER NOT NULL, 
    [COUNTRY] TEXT NOT NULL,
    [YEAR] INTEGER NOT NULL,
    [VALUE] REAL,
    FOREIGN KEY(COUNTRY) REFERENCES Country(COUNTRY),
    FOREIGN KEY(YEAR) REFERENCES YEAR(YEAR),
    FOREIGN KEY(HSCODE) REFERENCES Commodity(HSCODE)
);
"""
create_table(conn_norm, create_table_sql)

In [27]:
sql_statement = 'SELECT Commodity, country, year, value from export'
values = execute_sql_statement(sql_statement, conn)
print(values[:10])

[('MEAT AND EDIBLE MEAT OFFAL.', 'AFGHANISTAN TIS', 2018, 0.18), ('FISH AND CRUSTACEANS, MOLLUSCS AND OTHER AQUATIC INVERTABRATES.', 'AFGHANISTAN TIS', 2018, 0.0), ("DAIRY PRODUCE; BIRDS' EGGS; NATURAL HONEY; EDIBLE PROD. OF ANIMAL ORIGIN, NOT ELSEWHERE SPEC. OR INCLUDED.", 'AFGHANISTAN TIS', 2018, 12.48), ('LIVE TREES AND OTHER PLANTS; BULBS; ROOTS AND THE LIKE; CUT FLOWERS AND ORNAMENTAL FOLIAGE.', 'AFGHANISTAN TIS', 2018, 0.0), ('EDIBLE VEGETABLES AND CERTAIN ROOTS AND TUBERS.', 'AFGHANISTAN TIS', 2018, 1.89), ('EDIBLE FRUIT AND NUTS; PEEL OR CITRUS FRUIT OR MELONS.', 'AFGHANISTAN TIS', 2018, 25.01), ('COFFEE, TEA, MATE AND SPICES.', 'AFGHANISTAN TIS', 2018, 13.75), ('CEREALS.', 'AFGHANISTAN TIS', 2018, 0.75), ('PRODUCTS OF THE MILLING INDUSTRY; MALT; STARCHES; INULIN; WHEAT GLUTEN.', 'AFGHANISTAN TIS', 2018, 0.01), ('OIL SEEDS AND OLEA. FRUITS; MISC. GRAINS, SEEDS AND FRUIT; INDUSTRIAL OR MEDICINAL PLANTS; STRAW AND FODDER.', 'AFGHANISTAN TIS', 2018, 10.02)]


In [28]:
sql_statement = 'SELECT HSCODE, COMMODITY from COMMODITY'
values = execute_sql_statement(sql_statement, conn_norm)
print(values[:10])

[(1, 'AIRCRAFT, SPACECRAFT, AND PARTS THEREOF.'), (2, 'ALBUMINOIDAL SUBSTANCES; MODIFIED STARCHES; GLUES; ENZYMES.'), (3, 'ALUMINIUM AND ARTICLES THEREOF.'), (4, 'ANIMAL OR VEGETABLE FATS AND OILS AND THEIR CLEAVAGE PRODUCTS; PRE. EDIBLE FATS; ANIMAL OR VEGETABLE WAXEX.'), (5, 'ARMS AND AMMUNITION; PARTS AND ACCESSORIES THEREOF.'), (6, 'ARTICLES OF APPAREL AND CLOTHING ACCESSORIES, KNITTED OR CORCHETED.'), (7, 'ARTICLES OF APPAREL AND CLOTHING ACCESSORIES, NOT KNITTED OR CROCHETED.'), (8, 'ARTICLES OF IRON OR STEEL'), (9, 'ARTICLES OF LEATHER,SADDLERY AND HARNESS;TRAVEL GOODS, HANDBAGS AND SIMILAR CONT.ARTICLES OF ANIMAL GUT(OTHR THN SILK-WRM)GUT.'), (10, 'ARTICLES OF STONE, PLASTER, CEMENT, ASBESTOS, MICA OR SIMILAR MATERIALS.')]


In [30]:
commodity_lookup = {}
for hscode, commodity in values:
    commodity_lookup[commodity] = hscode
    
print(commodity_lookup)

{'AIRCRAFT, SPACECRAFT, AND PARTS THEREOF.': 1, 'ALBUMINOIDAL SUBSTANCES; MODIFIED STARCHES; GLUES; ENZYMES.': 2, 'ALUMINIUM AND ARTICLES THEREOF.': 3, 'ANIMAL OR VEGETABLE FATS AND OILS AND THEIR CLEAVAGE PRODUCTS; PRE. EDIBLE FATS; ANIMAL OR VEGETABLE WAXEX.': 4, 'ARMS AND AMMUNITION; PARTS AND ACCESSORIES THEREOF.': 5, 'ARTICLES OF APPAREL AND CLOTHING ACCESSORIES, KNITTED OR CORCHETED.': 6, 'ARTICLES OF APPAREL AND CLOTHING ACCESSORIES, NOT KNITTED OR CROCHETED.': 7, 'ARTICLES OF IRON OR STEEL': 8, 'ARTICLES OF LEATHER,SADDLERY AND HARNESS;TRAVEL GOODS, HANDBAGS AND SIMILAR CONT.ARTICLES OF ANIMAL GUT(OTHR THN SILK-WRM)GUT.': 9, 'ARTICLES OF STONE, PLASTER, CEMENT, ASBESTOS, MICA OR SIMILAR MATERIALS.': 10, 'BEVERAGES, SPIRITS AND VINEGAR.': 11, 'CARPETS AND OTHER TEXTILE FLOOR COVERINGS.': 12, 'CERAMIC PRODUCTS.': 13, 'CEREALS.': 14, 'CLOCKS AND WATCHES AND PARTS THEREOF.': 15, 'COCOA AND COCOA PREPARATIONS.': 16, 'COFFEE, TEA, MATE AND SPICES.': 17, 'COPPER AND ARTICLES THEREOF.'

In [31]:
commodity_lookup['MEAT AND EDIBLE MEAT OFFAL.']

45

In [33]:
# create_table_sql = """CREATE TABLE IF NOT EXISTS [Export] (
#     [VALUEID] INTEGER NOT NULL PRIMARY KEY,
#     [HSCODE] INTEGER NOT NULL, 
#     [COUNTRY] TEXT NOT NULL,
#     [YEAR] INTEGER NOT NULL,
#     [VALUE] REAL,
#     FOREIGN KEY(COUNTRY) REFERENCES Country(COUNTRY),
#     FOREIGN KEY(YEAR) REFERENCES YEAR(YEAR),
#     FOREIGN KEY(HSCODE) REFERENCES Commodity(HSCODE)
# );
# """

sql_statement = 'SELECT Commodity, country, year, value from export'
values = execute_sql_statement(sql_statement, conn)
# with conn_norm:
count = 0
with conn_norm:
    for value in values:
        text = value[0]
        hscode = commodity_lookup[text]
        insert_tuple = (hscode, value[1], value[2], value[3])
        insert_value(conn_norm, insert_tuple)


In [37]:
sql_statement = """SELECT 
export.hscode,
Commodity.COMMODITY,
export.country,
export.year,
export.value
FROM Export
JOIN Commodity ON Commodity.hscode = export.hscode
"""
df = pd.read_sql_query(sql_statement, conn_norm)
display(df)

,HSCODE,COMMODITY,COUNTRY,YEAR,VALUE
0,45,MEAT AND EDIBLE MEAT OFFAL.,AFGHANISTAN TIS,2018,0.18
1,28,"FISH AND CRUSTACEANS, MOLLUSCS AND OTHER AQUAT...",AFGHANISTAN TIS,2018,0.00
2,21,DAIRY PRODUCE; BIRDS' EGGS; NATURAL HONEY; EDI...,AFGHANISTAN TIS,2018,12.48
3,41,LIVE TREES AND OTHER PLANTS; BULBS; ROOTS AND ...,AFGHANISTAN TIS,2018,0.00
4,23,EDIBLE VEGETABLES AND CERTAIN ROOTS AND TUBERS.,AFGHANISTAN TIS,2018,1.89
...,...,...,...,...,...
137018,90,"TOYS, GAMES AND SPORTS REQUISITES; PARTS AND A...",ZIMBABWE,2010,0.03
137019,51,MISCELLANEOUS MANUFACTURED ARTICLES.,ZIMBABWE,2010,0.45
137020,97,WORKS OF ART COLLECTORS' PIECES AND ANTIQUES.,ZIMBABWE,2010,0.00
137021,74,PROJECT GOODS; SOME SPECIAL USES.,ZIMBABWE,2010,0.00


In [46]:
conn_norm.close()

In [42]:
import time
time.monotonic() 



270581.031

In [44]:

conn_norm = create_connection('india_export_norm.db')
cur = conn_norm.cursor()
cur.execute('DROP TABLE EXPORT')
create_table_sql = """CREATE TABLE IF NOT EXISTS [Export] (
    [VALUEID] INTEGER NOT NULL PRIMARY KEY,
    [HSCODE] INTEGER NOT NULL, 
    [COUNTRY] TEXT NOT NULL,
    [YEAR] INTEGER NOT NULL,
    [VALUE] REAL,
    FOREIGN KEY(COUNTRY) REFERENCES Country(COUNTRY),
    FOREIGN KEY(YEAR) REFERENCES YEAR(YEAR),
    FOREIGN KEY(HSCODE) REFERENCES Commodity(HSCODE)
);
"""
create_table(conn_norm, create_table_sql)

with conn_norm:
    start_time = time.monotonic() 
    for value in values:
        text = value[0]
        hscode = commodity_lookup[text]
        insert_tuple = (hscode, value[1], value[2], value[3])
        insert_value(conn_norm, insert_tuple)
end_time = time.monotonic()     


print(end_time-start_time)

1.5


In [45]:
insert_tuples = []

for value in values:
    text = value[0]
    hscode = commodity_lookup[text]
    insert_tuple = (hscode, value[1], value[2], value[3])
    insert_tuples.append(insert_tuple)
print(insert_tuples[:100])

[(45, 'AFGHANISTAN TIS', 2018, 0.18), (28, 'AFGHANISTAN TIS', 2018, 0.0), (21, 'AFGHANISTAN TIS', 2018, 12.48), (41, 'AFGHANISTAN TIS', 2018, 0.0), (23, 'AFGHANISTAN TIS', 2018, 1.89), (22, 'AFGHANISTAN TIS', 2018, 25.01), (17, 'AFGHANISTAN TIS', 2018, 13.75), (14, 'AFGHANISTAN TIS', 2018, 0.75), (73, 'AFGHANISTAN TIS', 2018, 0.01), (56, 'AFGHANISTAN TIS', 2018, 10.02), (38, 'AFGHANISTAN TIS', 2018, 0.04), (92, 'AFGHANISTAN TIS', 2018, None), (4, 'AFGHANISTAN TIS', 2018, 0.37), (85, 'AFGHANISTAN TIS', 2018, 15.92), (16, 'AFGHANISTAN TIS', 2018, 0.0), (67, 'AFGHANISTAN TIS', 2018, 0.63), (69, 'AFGHANISTAN TIS', 2018, 5.04), (49, 'AFGHANISTAN TIS', 2018, 3.08), (11, 'AFGHANISTAN TIS', 2018, 0.01), (78, 'AFGHANISTAN TIS', 2018, 0.22), (88, 'AFGHANISTAN TIS', 2018, 91.22), (80, 'AFGHANISTAN TIS', 2018, 0.02), (46, 'AFGHANISTAN TIS', 2018, 0.59), (35, 'AFGHANISTAN TIS', 2018, 1.08), (59, 'AFGHANISTAN TIS', 2018, 7.15), (64, 'AFGHANISTAN TIS', 2018, 80.27), (27, 'AFGHANISTAN TIS', 2018, 0.0)

In [47]:

conn_norm = create_connection('india_export_norm.db')
cur = conn_norm.cursor()
cur.execute('DROP TABLE EXPORT')
create_table_sql = """CREATE TABLE IF NOT EXISTS [Export] (
    [VALUEID] INTEGER NOT NULL PRIMARY KEY,
    [HSCODE] INTEGER NOT NULL, 
    [COUNTRY] TEXT NOT NULL,
    [YEAR] INTEGER NOT NULL,
    [VALUE] REAL,
    FOREIGN KEY(COUNTRY) REFERENCES Country(COUNTRY),
    FOREIGN KEY(YEAR) REFERENCES YEAR(YEAR),
    FOREIGN KEY(HSCODE) REFERENCES Commodity(HSCODE)
);
"""
create_table(conn_norm, create_table_sql)
sql = ''' INSERT INTO Export (HSCODE, COUNTRY, YEAR, VALUE)
              VALUES(?, ?, ?, ?) '''
cur = conn_norm.cursor()
start_time = time.monotonic() 
cur.executemany(sql, insert_tuples)
conn_norm.commit()
end_time = time.monotonic()     

In [48]:
print(end_time-start_time)

0.9219999999622814


In [ ]:
x = 'abcd'
x.upper

In [ ]:
conn_norm.commit